In [1]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
import plotly.graph_objects as go

from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [2]:
analysis_pkl = 'data/some_spectra/SI06122_result.pkl'
analysis_df = pd.read_pickle(analysis_pkl)

In [8]:
analysis_df.reset_index(drop=True, inplace=True)

In [9]:
analysis_df['test'] = analysis_df['decay_%']*analysis_df['intensity']/100

In [12]:
analysis_df

,nuclide_name,centroids,rough_sums,variances,recalib_energy,energy,intensity,unc_i,decay_%,is_key_gamma,nuclide_name,disintegr,activity_conc,test
0,212Pb,396.426778,630.310441,10404.641509,238.770932,238.6320,43.60000,0.5307178534777213,100.000,True,212Pb,7.324243e+04,14.648486,43.600000
1,214Pb,488.967375,249.976688,4072.023312,295.305981,295.2240,18.47088,0.11249527437185973,100.000,False,214Pb,8.502770e+04,17.005540,18.470880
2,214Pb,581.647407,430.396173,5405.821287,351.925344,351.9320,35.72370,0.2352736185805795,100.000,True,214Pb,8.929519e+04,17.859037,35.723700
3,208Tl,960.065118,483.773366,3335.841582,583.096300,583.1870,85.00000,0.3,100.000,True,208Tl,6.280187e+04,12.560374,85.000000
4,214Bi,1002.931019,769.886444,3458.113556,609.281662,609.3210,45.44000,0.19,99.979,True,214Bi,1.925320e+05,38.506404,45.430458
5,110ag,1082.200436,2125.483990,4570.516010,657.704238,657.7600,95.61000,0.09,98.670,True,110ag,2.655030e+05,53.100602,94.338387
6,137cs,1088.105023,401.081225,2556.918775,661.311093,661.6570,85.10000,0.2,100.000,True,137cs,5.650064e+04,11.300127,85.100000
7,110ag,1115.022855,176.695976,1999.057501,677.753980,677.6217,10.70000,0.05,98.670,False,110ag,2.009797e+05,40.195943,10.557690
8,110ag,1161.943753,336.295814,2591.560129,706.415658,706.6760,16.69000,0.07,98.670,False,110ag,2.517501e+05,50.350025,16.468023
9,131i,1189.208186,160.747926,1865.322724,723.070063,722.9110,1.76855,0.026749,100.000,False,131i,1.151661e+06,230.332119,1.768550


In [18]:
analysis_df.reindex(['nuclide_name'])

,nuclide_name,centroids,rough_sums,variances,recalib_energy,energy,intensity,unc_i,decay_%,is_key_gamma,nuclide_name,disintegr,activity_conc,test
nuclide_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# d5_where = d5.where(d5.nuclide_name=="110ag")
# d5_where
d5_loc = analysis_df.loc[analysis_df.nuclide_name=="110ag"]
d5_loc

ValueError: Index data must be 1-dimensional

In [14]:
# 2023-Ago-4 PAREI AQUI. Fazer a indexação para o cálculo da atividade considerando a média das linhas de cada nuclídeo
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#dataframe-column-selection-in-groupby
d5_grouped = analysis_df.groupby("nuclide_name")[['energy','intensity','centroids','rough_sums','variances','disintegr']]
# d5_grouped = d5.groupby("nuclide_name")[['disintegr']]
# sdfsdf = pd.DataFrame([[d5_grouped.mean(), d5_grouped.median()]]) 
# sdfsdf


ValueError: Grouper for 'nuclide_name' not 1-dimensional

In [ ]:
d5_grouped.mean()

In [64]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#built-in-aggregation-methods
d5_aggr_df = pd.DataFrame(df_result.groupby("nuclide_name")[['disintegr']].mean())
d5_aggr_df['dsnt_median'] = df_result.groupby("nuclide_name")[['disintegr']].median()
d5_aggr_df['dsnt_counts'] = df_result.groupby("nuclide_name")[['disintegr']].count()
d5_aggr_df['dsnt_sem'] = df_result.groupby("nuclide_name")[['disintegr']].sem()
d5_aggr_df['dsnt_std'] = df_result.groupby("nuclide_name")[['disintegr']].std()
d5_aggr_df

ValueError: Grouper for 'nuclide_name' not 1-dimensional

In [ ]:
df_result.groupby("nuclide_name")[['disintegr']].describe()

In [ ]:
d5_aggr_df['disint_mean'] = d5_grouped.mean()
d5_aggr_df['disint_count'] = d5_grouped.count()
d5_aggr_df

In [ ]:
df_result

In [ ]:
# 2023-May-24
# Implementing:
#  - nuclide labels
# https://plotly.com/python/text-and-annotations/
#  - and energy search in pandas
# https://stackoverflow.com/questions/63529555/get-list-of-occurrences-using-pandas

In [ ]:
# counts_nzero_cps = counts_nzero / a_spec.lv_time
# unc_y_cps = unc_y / a_spec.lv_time
# fin_bl_cps = fin_bl / a_spec.lv_time
# counts_cps = counts / a_spec.lv_time
# en_kev_nzero = get_energy(chans_nzero)
# en_kev = get_energy(chans)
# peaks_net_kev = get_energy(peaks_net)

In [ ]:
help(spec_engy_identif)

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [11]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [12]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [13]:
result.loc[result.A=='A1'].loc[result.C=='C0']

,,A,B,C,D


In [ ]:
# https://stackoverflow.com/questions/63529555

In [ ]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [ ]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

In [ ]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

In [ ]:
# The End